# Using `kugupu` to calculate molecular coupling networks WITH ML

This notebook demonstrates how to calculate molecular coupling between fragments, inspect the results and save and load these results to file.  These results files will be the basis of all further analysis done using the `kugupu` package.

This will require version 0.20.0 of MDAnalysis, and kugupu to be installed.

In [1]:
import MDAnalysis as mda
import numpy as np
import kugupu as kgp

models available
{'ocelotml': <class 'kugupu.ocelotl_model.OcelotMLModel'>, 'yaehmop': <class 'kugupu.yaehmop.YaehmopModel'>}


Firstly we create an `MDAnalysis.Universe` object from our simulation files:

In [2]:
u = mda.Universe('datafiles/C6.data', 'datafiles/C6.dcd')

/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


This system has 46,500 atoms in 250 different fragments.

In [3]:
print(u.atoms.n_atoms, len(u.atoms.fragments))

46500 250


Our dynamics simulation has 5 frames of results.

In [4]:
print(u.trajectory.n_frames)

5


In [5]:
# u.atoms.fragments[0].names

To perform the coupling calculations our `Universe` will require bond information (for determining fragments) and element information (for the tight binding calculations) stored inside the `.names` attribute.

Our Lammps Data file did not include element symbols, so we can add these to the Universe now...

In [6]:
def add_names(u):
    # Guesses atom names based upon masses
    def approx_equal(x, y):
        return abs(x - y) < 0.1
    
    # mapping of atom mass to element
    massdict = {}
    for m in set(u.atoms.masses):
        for elem, elem_mass in mda.guesser.tables.masses.items():
            if approx_equal(m, elem_mass):
                massdict[m] = elem
                break
        else:
            raise ValueError
            
    u.add_TopologyAttr('names')
    for m, e in massdict.items():
        u.atoms[u.atoms.masses == m].names = e
        

add_names(u)

In [11]:
from openff.toolkit.topology import Molecule
from MDAnalysis.topology.guessers import guess_types
from kugupu._yaehmop import shift_dimer_images
from MDAnalysis.lib.mdamath import triclinic_vectors
from kugupu.dimers import find_dimers
from MDAnalysis.transformations.wrap import unwrap

TEST = 200

elements = guess_types(u.atoms.names)
u.add_TopologyAttr("elements", elements)

ag = u.atoms
# transform = mda.transformations.unwrap(ag)
# u.trajectory.add_transformations(transform)

# print(u.dimensions.shape)
example_fragments = u.atoms.fragments
dimers_list = find_dimers(example_fragments, cutoff=5.0)
list_dimers = sorted(dimers_list.items())
frag_i, frag_j =list_dimers[TEST][-1]
shifted_coords = shift_dimer_images(frag_i, frag_j)
total_frag = frag_i + frag_j
total_frag.positions = shifted_coords
rdkit = total_frag.convert_to.rdkit()
mol = Molecule.from_rdkit(rdkit, allow_undefined_stereo=True)
mol


/var/folders/8_/xls29m695yl7qglq81r94h7w0000gr/T/ipykernel_98637/1284948650.py:10: DeprecationWarning: `guess_types` is deprecated!
`guess_types` will be removed in release 3.0.0.
MDAnalysis.topology.guessers is deprecated in favour of the new Guessers API. See MDAnalysis.guesser.default_guesser for more details.
  elements = guess_types(u.atoms.names)
2025-06-11T10:17:14.883796+0100 INFO Finding dimers within 5.0, passed 250 fragments
2025-06-11T10:17:15.243097+0100 INFO Found 3282 dimers


shift


NGLWidget()

In [16]:
from qcportal.singlepoint import SinglepointDriver
from openff.qcsubmit.factories import BasicDatasetFactory
from openff.qcsubmit.common_structures import QCSpec
from qcelemental.models.results import WavefunctionProtocolEnum
from qcportal import PortalClient


USERNAME = "charlie"
PASSWORD = "kuano123"
ADDRESS ="http://localhost:7777"

client = PortalClient(
    address=ADDRESS, username = USERNAME, password= PASSWORD
)



In [18]:
factory = BasicDatasetFactory(
    driver=SinglepointDriver.energy,
    qc_specifications={
        "xtb": QCSpec(
            program="xtb",
            method="gfn0xtb",
            basis=None,
            spec_name="ani1ccx",
            spec_description="ANI1ccx standard specification",
            store_wavefunction=WavefunctionProtocolEnum.all
        ),
    },
)
molecules = [mol]
factory.compute_tag = 'xtb'
dataset = factory.create_dataset(
    dataset_name="xtb_test",
    description="xtb test for coupling",
    tagline="test dataset for xtb wavefunction",
    molecules=molecules,  #I deleted
)
dataset.submit(
    client = client,
)

Preparation                   :   0%|                     | 0/1 [00:00<?, ?it/s]/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/openff/qcsubmit/utils/smirnoff.py:196: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  new_conformer[index_mapping[i]] = conformer[i]
Preparation                   : 100%|█████████████| 1/1 [00:00<00:00,  2.31it/s]
'metadata' parameter has been deprecated and will be removed in a future version. Use 'extras' instead
'tag' is deprecated; use 'compute_tag' instead
'priority' is deprecated; use 'compute_priority' instead


InsertCountsMetadata(n_inserted=1, n_existing=0, error_description=None, errors=[])

In [ ]:
c1 = frag_i.center_of_mass()

In [22]:
c2 = frag_j.center_of_mass()

In [24]:
from MDAnalysis.lib import distances

d1 = distances.calc_bonds(c1,c2)
d2 = distances.calc_bonds(c1, c2, frag_i.dimensions)
d1-d2

-5.332449681816342e-07

In [25]:
frag_i.dimensions

array([82.370605, 82.370605, 82.370605, 90.      , 90.      , 90.      ],
      dtype=float32)

In [134]:
example_fragment.atoms.positions.shape

(186, 3)

In [37]:
u.dimensions

array([82.370605, 82.370605, 82.370605, 90.      , 90.      , 90.      ],
      dtype=float32)

In [23]:
from io import StringIO

output = StringIO()
# example_fragment.write(filename= output.getvalue(), format='.pdb')
output.name

AttributeError: '_io.StringIO' object has no attribute 'name'

In [29]:
type(example_fragment.atoms)

MDAnalysis.core.groups.AtomGroup

In [13]:
coords

array([[ 97.48227 ,  82.034355, 131.86044 ],
       [ 97.95442 ,  82.892746, 130.86584 ],
       [ 99.27297 ,  82.63595 , 130.42174 ],
       [100.05624 ,  81.52764 , 130.84343 ],
       [ 99.55603 ,  80.64784 , 131.74911 ],
       [ 98.20787 ,  80.84279 , 132.20345 ],
       [ 96.50943 ,  82.24995 , 132.29575 ],
       [101.07468 ,  81.39984 , 130.50264 ],
       [ 97.2372  ,  84.12135 , 130.31897 ],
       [ 98.471245,  84.59264 , 129.4375  ],
       [ 99.55969 ,  83.72523 , 129.5342  ],
       [ 98.73449 ,  85.74673 , 128.67285 ],
       [100.106346,  85.785576, 128.24309 ],
       [100.981865,  84.35223 , 128.77982 ],
       [100.21008 ,  79.42629 , 132.36818 ],
       [ 98.98248 ,  78.95351 , 133.22961 ],
       [ 97.877556,  79.72083 , 133.02733 ],
       [ 98.652054,  77.95627 , 134.1653  ],
       [ 97.29675 ,  77.97778 , 134.62914 ],
       [ 96.408104,  79.2198  , 133.90623 ],
       [100.44462 ,  86.95385 , 127.66395 ],
       [ 97.914085,  87.267555, 128.25804 ],
       [ 9

## Running the coupling matrix calculation

The coupling matrix between fragments is calculated using the `kgp.coupling_matrix` function.

Here we are calculating the coupling matrix for fragments in the Universe `u` where
- coupling is calculated between fragments with a closest approach of less than 5.0 Angstrom (`nn_cutoff`)
- coupling is calculated between the LUMO upwards (`state='lumo'`)
- one state per fragment is considered (`degeneracy=1`)
- we will analyse up to frame 3 (`stop=3`)

This function will (for each frame)
- identify which fragments are close enough to possibly be electronically coupled
- run a tight binding calculation between all pairs identified
- calculate the molecular coupling based on this tight binding calculation

In [ ]:
res = kgp.coupling_matrix(u, nn_cutoff=5.0, state='lumo', degeneracy=1, stop=3, model='ocelotml', client=True)

/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60825 instead
  warnings.warn(
2025-06-05T16:08:09.655317+0100 INFO Processing 3 frames
2025-06-05T16:08:09.657143+0100 INFO Processing frame 1 of 3
/Users/k2584788/.local/share/mamba/envs/forked_kugupu/lib/python3.10/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"
2025-06-05T16:08:10.030477+0100 INFO Finding dimers within 5.0, passed 250 fragments
2025-06-05T16:08:10.387019+0100 INF

The `res` object is a namedtuple which contains all the data necessary to perform further analysis.
This object has various attributes which will not be briefly explained.

The `.frames` attribute records which frames from the trajectory were analysed.
This is useful to later cross reference data with the original MD trajectory data.

In [ ]:
print(res.frames)

[0 1 2]


The `.degeneracy` attribute stores how many degenerate states were considered for each fragment.
This value will not change over time, so this array has shape `nfragments`.

In this example only a single state per fragment was considered. 

In [ ]:
print(res.degeneracy)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


The `.H_frag` attribute contains the molecular coupling values, stored inside a 3d numpy array.
The first dimension is along the number of frames (quasi time axis),
while the other two move along fragments in the system.

For example `res.H_frag[0, 1, 71]` gives the coupling (in eV) between the 2nd and 13th fragments in the first frame.

In [ ]:
print(res.H_frag.shape)

print(res.H_frag[2, 1, 71])

(3, 250, 250)
0.0002765886942250739


Producing these results is often a time consuming part of the analysis,
therefore it is wise to save them to a file so you can come back to them later!

This can be done using the `kugupu.save_results` function, which will save the results to a hdf5 (compressed) format.

In [ ]:
kgp.save_results('myresults.hdf5', res)

These results can then be retrieved again using the `kugupu.load_results` function:

In [ ]:
kgp.load_results('./myresults.hdf5')

KugupuResults(frames=array([0, 1, 2]), H_frag=array([[[-10.27936597,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        , -10.32038834,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        , -10.35344287, ...,   0.        ,
           0.        ,   0.        ],
        ...,
        [  0.        ,   0.        ,   0.        , ..., -10.43146138,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
         -10.50477574,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        , -10.37584228]],

       [[-10.38898008,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        , -10.43337746,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        , -10.44523772, ...,   0.        ,
     